In [1]:
# Importing libraries
import os
import logging
import pandas as pd
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from typing import Tuple, List, Dict, Optional
import time

# LangChain imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.documents import Document
from langchain_core.tools import tool

# Gradio for UI
import gradio as gr

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Logging setup
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(name)s - %(message)s"
LOG_DATE_FORMAT = "%Y-%m-%d %H:%M:%S"
log_filename = os.path.join(LOG_DIR, f"chatbot_{datetime.now().strftime('%Y%m%d')}.log")

file_handler = logging.FileHandler(log_filename, encoding='utf-8')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter(LOG_FORMAT, datefmt=LOG_DATE_FORMAT))

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter(LOG_FORMAT, datefmt=LOG_DATE_FORMAT))

logger = logging.getLogger("MovieChatbot")
logger.setLevel(logging.DEBUG)
logger.addHandler(file_handler)
logger.addHandler(console_handler)
logger.propagate = False

logger.info("=" * 50)
logger.info("IMDb Movie Chatbot - Session Started")
logger.info("=" * 50)

print("All libraries imported successfully!")
print(f"Logging to: {log_filename}")

2026-01-18 14:40:26 - INFO - MovieChatbot - ==================================================


2026-01-18 14:40:26 - INFO - MovieChatbot - IMDb Movie Chatbot - Session Started


2026-01-18 14:40:26 - INFO - MovieChatbot - ==================================================


All libraries imported successfully!
Logging to: logs\chatbot_20260118.log


In [2]:
# Store your OpenAI API key
# Option 1: Load from .env file (recommended for security)
load_dotenv()

# Option 2: Set directly (use for testing only - don't commit to version control)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Verify API key is set
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY:
    print("OpenAI API key loaded successfully!")
else:
    print("WARNING: OPENAI_API_KEY not found. Please set it in .env file or environment variables.")
    print("Create a .env file with: OPENAI_API_KEY=your-key-here")

OpenAI API key loaded successfully!


In [3]:
# Load the data
# Dataset path - adjust if loading from Google Drive
DATASET_PATH = "IMDb_Dataset (1).csv"

logger.info(f"Loading dataset from: {DATASET_PATH}")

# Load the IMDb dataset
try:
    df = pd.read_csv(DATASET_PATH)
    logger.info(f"Dataset loaded successfully: {df.shape[0]} movies, {df.shape[1]} features")
except FileNotFoundError:
    logger.error(f"Dataset file not found: {DATASET_PATH}")
    raise
except Exception as e:
    logger.error(f"Error loading dataset: {str(e)}")
    raise

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} movies, {df.shape[1]} features")
print(f"\nColumns: {list(df.columns)}")

2026-01-18 14:40:26 - INFO - MovieChatbot - Loading dataset from: IMDb_Dataset (1).csv


2026-01-18 14:40:26 - INFO - MovieChatbot - Dataset loaded successfully: 3173 movies, 10 features


Dataset loaded successfully!
Shape: 3173 movies, 10 features

Columns: ['Title', 'IMDb Rating', 'Year', 'Certificates', 'Genre', 'Director', 'Star Cast', 'MetaScore', 'Poster-src', 'Duration (minutes)']


In [4]:
# View & Understand the data

# Basic info
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)

# First few rows
print("\n📊 Sample Data (First 5 rows):")
display(df.head())

# Data types and non-null counts
print("\n📋 Data Types & Missing Values:")
print(df.info())

# Statistical summary for numerical columns
print("\n📈 Statistical Summary (Numerical):")
display(df.describe())

# Missing values analysis
print("\n⚠️ Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})
display(missing_df[missing_df['Missing Count'] > 0])

# Unique values for categorical columns
print("\n🎭 Unique Values:")
print(f"- Genres: {df['Genre'].nunique()} unique genres")
print(f"- Certificates: {df['Certificates'].nunique()} unique certificates")
print(f"- Directors: {df['Director'].nunique()} unique directors")
print(f"- Year range: {df['Year'].min()} - {df['Year'].max()}")

# Genre distribution (top 10)
print("\n🎬 Top 10 Genres:")
print(df['Genre'].value_counts().head(10))

# Rating distribution
print("\n⭐ Rating Distribution:")
print(f"- Mean IMDb Rating: {df['IMDb Rating'].mean():.2f}")
print(f"- Median IMDb Rating: {df['IMDb Rating'].median():.2f}")
print(f"- Rating Range: {df['IMDb Rating'].min()} - {df['IMDb Rating'].max()}")

DATASET OVERVIEW

📊 Sample Data (First 5 rows):


,Title,IMDb Rating,Year,Certificates,Genre,Director,Star Cast,MetaScore,Poster-src,Duration (minutes)
0,End of the Spear,6.8,2005,PG-13,Adventure,Jim Hanon,Louie LeonardoChad AllenJack Guzman,45.0,https://m.media-amazon.com/images/M/MV5BMTYxOT...,108.0
1,Elvira Madigan,7.0,1967,PG,Biography,Bo Widerberg,Pia DegermarkThommy BerggrenLennart Malmer,66.0,https://m.media-amazon.com/images/M/MV5BMmY2Nj...,91.0
2,The Kid Stays in the Picture,7.3,2002,R,Documentary,Nanette Burstein,Robert EvansEddie AlbertPeter Bart,75.0,https://m.media-amazon.com/images/M/MV5BZjhiZm...,93.0
3,It Ain't Over,8.2,2022,PG,Documentary,Sean Mullin,Andy AndresRoger AngellMarty Appel,79.0,https://m.media-amazon.com/images/M/MV5BZWViYW...,99.0
4,Mahler,7.0,1974,PG,Biography,Ken Russell,Robert PowellGeorgina HaleLee Montague,66.0,https://m.media-amazon.com/images/M/MV5BYzY4Mz...,115.0



📋 Data Types & Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173 entries, 0 to 3172
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               3173 non-null   object 
 1   IMDb Rating         3173 non-null   float64
 2   Year                3173 non-null   int64  
 3   Certificates        3173 non-null   object 
 4   Genre               3173 non-null   object 
 5   Director            3173 non-null   object 
 6   Star Cast           3173 non-null   object 
 7   MetaScore           3173 non-null   float64
 8   Poster-src          3173 non-null   object 
 9   Duration (minutes)  3173 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 248.0+ KB
None

📈 Statistical Summary (Numerical):


,IMDb Rating,Year,MetaScore,Duration (minutes)
count,3173.000000,3173.000000,3173.000000,3173.000000
mean,6.931012,2004.958399,65.993066,116.304822
std,0.805202,16.819985,11.358619,22.123032
min,3.900000,1917.000000,20.000000,46.000000
25%,6.400000,1997.000000,64.000000,105.000000
50%,6.900000,2010.000000,66.000000,116.300000
75%,7.500000,2017.000000,69.000000,122.000000
max,9.500000,2025.000000,100.000000,317.000000



⚠️ Missing Values:


,Missing Count,Percentage



🎭 Unique Values:
- Genres: 17 unique genres
- Certificates: 16 unique certificates
- Directors: 2089 unique directors
- Year range: 1917 - 2025

🎬 Top 10 Genres:
Genre
Biography      868
Action         556
Drama          416
Documentary    251
Comedy         237
Animation      215
Sci-Fi         203
Adventure      202
Horror          93
Crime           66
Name: count, dtype: int64

⭐ Rating Distribution:
- Mean IMDb Rating: 6.93
- Median IMDb Rating: 6.90
- Rating Range: 3.9 - 9.5


In [5]:
# Create movie description for each movie from the details provided in the dataset

def create_movie_description(row):
    """
    Create a rich text description for each movie combining all available metadata.
    This description will be used for embedding and retrieval.
    """
    # Handle missing values
    title = row['Title'] if pd.notna(row['Title']) else 'Unknown Title'
    year = int(row['Year']) if pd.notna(row['Year']) else 'Unknown Year'
    genre = row['Genre'] if pd.notna(row['Genre']) else 'Unknown Genre'
    director = row['Director'] if pd.notna(row['Director']) else 'Unknown Director'
    cast = row['Star Cast'] if pd.notna(row['Star Cast']) else 'Unknown Cast'
    rating = row['IMDb Rating'] if pd.notna(row['IMDb Rating']) else 'N/A'
    metascore = row['MetaScore'] if pd.notna(row['MetaScore']) else 'N/A'
    certificate = row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated'
    duration = int(row['Duration (minutes)']) if pd.notna(row['Duration (minutes)']) else 'Unknown'
    poster = row['Poster-src'] if pd.notna(row['Poster-src']) else ''
    
    # Create structured description
    description = f"""
Movie Title: {title}
Year: {year}
Genre: {genre}
Director: {director}
Star Cast: {cast}
IMDb Rating: {rating}/10
MetaScore: {metascore}
Certificate: {certificate}
Duration: {duration} minutes
Poster URL: {poster}

This is a {genre} movie titled "{title}" released in {year}. 
It was directed by {director} and stars {cast}. 
The film has an IMDb rating of {rating}/10 and a MetaScore of {metascore}. 
It is rated {certificate} with a runtime of {duration} minutes.
""".strip()
    
    return description

# Apply the function to create descriptions
print("Creating movie descriptions...")
df['description'] = df.apply(create_movie_description, axis=1)

# Preview a sample description
print("\n✅ Movie descriptions created successfully!")
print(f"\nSample description for first movie:\n")
print("-" * 60)
print(df['description'].iloc[0])
print("-" * 60)

# Show description statistics
print(f"\nDescription Statistics:")
print(f"- Total movies with descriptions: {len(df)}")
print(f"- Average description length: {df['description'].str.len().mean():.0f} characters")

Creating movie descriptions...



✅ Movie descriptions created successfully!

Sample description for first movie:

------------------------------------------------------------
Movie Title: End of the Spear
Year: 2005
Genre: Adventure
Director: Jim Hanon
Star Cast: Louie LeonardoChad AllenJack Guzman
IMDb Rating: 6.8/10
MetaScore: 45.0
Certificate: PG-13
Duration: 108 minutes
Poster URL: https://m.media-amazon.com/images/M/MV5BMTYxOTU0NDUzMV5BMl5BanBnXkFtZTcwNDgzMTczMQ@@._V1_QL75_UX190_CR0,2,190,281_.jpg

This is a Adventure movie titled "End of the Spear" released in 2005. 
It was directed by Jim Hanon and stars Louie LeonardoChad AllenJack Guzman. 
The film has an IMDb rating of 6.8/10 and a MetaScore of 45.0. 
It is rated PG-13 with a runtime of 108 minutes.
------------------------------------------------------------

Description Statistics:
- Total movies with descriptions: 3173
- Average description length: 623 characters


In [6]:
# Now, data is ready!
# Its time to create your vector store
# Perform Text Chunking

# For movie data, each movie description is a natural document unit
# We'll create Document objects with metadata for better retrieval

def create_documents_from_dataframe(df):
    """
    Convert DataFrame rows to LangChain Document objects with metadata.
    Each movie becomes a single document for optimal retrieval.
    """
    documents = []
    
    for idx, row in df.iterrows():
        # Create metadata for filtering and display
        metadata = {
            'title': row['Title'] if pd.notna(row['Title']) else 'Unknown',
            'year': int(row['Year']) if pd.notna(row['Year']) else 0,
            'genre': row['Genre'] if pd.notna(row['Genre']) else 'Unknown',
            'director': row['Director'] if pd.notna(row['Director']) else 'Unknown',
            'rating': float(row['IMDb Rating']) if pd.notna(row['IMDb Rating']) else 0.0,
            'certificate': row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated',
            'poster_url': row['Poster-src'] if pd.notna(row['Poster-src']) else '',
            'duration': int(row['Duration (minutes)']) if pd.notna(row['Duration (minutes)']) else 0,
            'index': idx
        }
        
        # Create Document with description as page_content
        doc = Document(
            page_content=row['description'],
            metadata=metadata
        )
        documents.append(doc)
    
    return documents

# Create documents
print("Converting movie data to documents...")
documents = create_documents_from_dataframe(df)

print(f"\n✅ Created {len(documents)} documents")
print(f"\nSample document:")
print("-" * 60)
print(f"Content preview: {documents[0].page_content[:200]}...")
print(f"\nMetadata: {documents[0].metadata}")
print("-" * 60)

# Optional: Use text splitter for very long documents
# For our movie descriptions, this isn't needed, but shown for completeness
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

# Check if any documents need splitting
long_docs = [d for d in documents if len(d.page_content) > 1000]
print(f"\nDocuments exceeding 1000 chars: {len(long_docs)}")

# For this dataset, we'll use documents as-is (each movie = 1 document)
# If needed: split_documents = text_splitter.split_documents(documents)

Converting movie data to documents...



✅ Created 3173 documents

Sample document:
------------------------------------------------------------
Content preview: Movie Title: End of the Spear
Year: 2005
Genre: Adventure
Director: Jim Hanon
Star Cast: Louie LeonardoChad AllenJack Guzman
IMDb Rating: 6.8/10
MetaScore: 45.0
Certificate: PG-13
Duration: 108 minute...

Metadata: {'title': 'End of the Spear', 'year': 2005, 'genre': 'Adventure', 'director': 'Jim Hanon', 'rating': 6.8, 'certificate': 'PG-13', 'poster_url': 'https://m.media-amazon.com/images/M/MV5BMTYxOTU0NDUzMV5BMl5BanBnXkFtZTcwNDgzMTczMQ@@._V1_QL75_UX190_CR0,2,190,281_.jpg', 'duration': 108, 'index': 0}
------------------------------------------------------------

Documents exceeding 1000 chars: 0


In [7]:
# Create embeddings for the chunks
# See https://python.langchain.com/docs/integrations/text_embedding/ for available models

# Initialize OpenAI Embeddings
# Using text-embedding-ada-002 (default) or text-embedding-3-small for better performance
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Cost-effective and performant
    # model="text-embedding-ada-002",  # Alternative option
)

print("✅ OpenAI Embeddings model initialized")
print(f"   Model: text-embedding-3-small")

# Test embedding on a sample text
sample_text = "Action movie with car chases"
sample_embedding = embeddings.embed_query(sample_text)

print(f"\n📐 Embedding dimensions: {len(sample_embedding)}")
print(f"   Sample embedding preview: [{sample_embedding[0]:.6f}, {sample_embedding[1]:.6f}, ...]")

✅ OpenAI Embeddings model initialized
   Model: text-embedding-3-small



📐 Embedding dimensions: 1536
   Sample embedding preview: [-0.025623, 0.044585, ...]


In [8]:
# Create a vector store using the created chunks and the embeddings model

VECTORSTORE_PATH = "imdb_vectorstore"

# Check if vector store already exists (faster loading on re-run)
if os.path.exists(VECTORSTORE_PATH):
    logger.info(f"Loading existing FAISS vector store from: {VECTORSTORE_PATH}")
    print("Loading existing FAISS vector store...")
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH, 
        embeddings,
        allow_dangerous_deserialization=True  # Required for loading
    )
    logger.info(f"Loaded vector store with {vectorstore.index.ntotal} vectors")
    print(f"✅ Loaded existing vector store with {vectorstore.index.ntotal} vectors")
else:
    logger.info(f"Creating new FAISS vector store from {len(documents)} documents")
    print("Creating FAISS vector store from movie documents...")
    print(f"This may take a few minutes for {len(documents)} documents...")
    
    # Create FAISS vector store from documents
    start_time = time.time() if 'time' in dir() else None
    vectorstore = FAISS.from_documents(
        documents=documents,
        embedding=embeddings
    )
    
    logger.info(f"Vector store created with {vectorstore.index.ntotal} vectors")
    print(f"\n✅ FAISS vector store created successfully!")
    print(f"   Total vectors: {vectorstore.index.ntotal}")
    
    # Save vector store locally for faster loading next time
    vectorstore.save_local(VECTORSTORE_PATH)
    logger.info(f"Vector store saved to: {VECTORSTORE_PATH}")
    print(f"   Vector store saved to: {VECTORSTORE_PATH}/")

# Test similarity search
logger.debug("Testing similarity search...")
print("\n🔍 Testing similarity search...")
test_query = "comedy movie with Jim Carrey"
similar_docs = vectorstore.similarity_search(test_query, k=3)

print(f"\nQuery: '{test_query}'")
print(f"Top 3 results:")
for i, doc in enumerate(similar_docs, 1):
    print(f"\n{i}. {doc.metadata['title']} ({doc.metadata['year']})")
    print(f"   Genre: {doc.metadata['genre']} | Rating: {doc.metadata['rating']}")

logger.debug(f"Similarity search test completed: {len(similar_docs)} results")

2026-01-18 14:40:30 - INFO - MovieChatbot - Loading existing FAISS vector store from: imdb_vectorstore


Loading existing FAISS vector store...


2026-01-18 14:40:30 - INFO - MovieChatbot - Loaded vector store with 3173 vectors


✅ Loaded existing vector store with 3173 vectors

🔍 Testing similarity search...



Query: 'comedy movie with Jim Carrey'
Top 3 results:

1. The Mask (1994)
   Genre: Action | Rating: 6.9

2. I Love You Phillip Morris (2009)
   Genre: Biography | Rating: 6.6

3. Man on the Moon (1999)
   Genre: Biography | Rating: 7.4


In [9]:
# Create the llm model

# Initialize ChatOpenAI with GPT-4 or GPT-3.5-turbo
# Note: streaming=True enables token-by-token streaming for real-time responses
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Cost-effective option with good performance
    # model="gpt-4o",     # Higher quality but more expensive
    # model="gpt-3.5-turbo",  # Budget option
    temperature=0.7,  # Balanced creativity
    max_tokens=1000,
    streaming=True,  # Enable streaming for real-time responses
)

# Create a non-streaming version for batch operations
llm_batch = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=1000,
    streaming=False,
)

logger.info("LLM models initialized (streaming + batch)")

print("✅ LLM model initialized")
print(f"   Model: gpt-4o-mini")
print(f"   Temperature: 0.7")
print(f"   Max tokens: 1000")
print(f"   Streaming: Enabled")

# Test the LLM
test_response = llm_batch.invoke("Say hello in one sentence.")
print(f"\n🧪 LLM test response: {test_response.content}")

2026-01-18 14:40:32 - INFO - MovieChatbot - LLM models initialized (streaming + batch)


✅ LLM model initialized
   Model: gpt-4o-mini
   Temperature: 0.7
   Max tokens: 1000
   Streaming: Enabled



🧪 LLM test response: Hello! How can I assist you today?


In [10]:
# Create the prompt template

# Define a comprehensive prompt template for movie recommendations
MOVIE_PROMPT_TEMPLATE = """You are an expert Movie Recommendation Assistant with access to the IMDb movie database. 
Your role is to help users discover movies based on their preferences and queries.

Use the following movie information from our database to answer the user's question:

{context}

Guidelines:
1. Only recommend movies from the provided context - do not make up movie information
2. Provide relevant details like title, year, genre, director, cast, and ratings when available
3. If the user asks for recommendations, suggest movies that match their criteria
4. If no relevant movies are found in the context, politely say so
5. Be conversational and helpful in your responses
6. Format your response clearly with movie details

User Question: {question}

Helpful Answer:"""

# Create the PromptTemplate object
prompt = PromptTemplate(
    template=MOVIE_PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

print("✅ Prompt template created")
print("\n📝 Template preview:")
print("-" * 60)
print(MOVIE_PROMPT_TEMPLATE[:500] + "...")

✅ Prompt template created

📝 Template preview:
------------------------------------------------------------
You are an expert Movie Recommendation Assistant with access to the IMDb movie database. 
Your role is to help users discover movies based on their preferences and queries.

Use the following movie information from our database to answer the user's question:

{context}

Guidelines:
1. Only recommend movies from the provided context - do not make up movie information
2. Provide relevant details like title, year, genre, director, cast, and ratings when available
3. If the user asks for recommendat...


In [11]:
# Create the document processing chain manually

class SimpleDocumentChain:
    def __init__(self, llm, prompt):
        self.llm = llm
        self.prompt = prompt

    def invoke(self, input_dict):
        context = input_dict.get("context", "")
        question = input_dict.get("input", "")

        # Format documents as context
        if isinstance(context, list):
            context_text = "\n\n".join([doc.page_content for doc in context])
        else:
            context_text = str(context)

        # Create the prompt
        formatted_prompt = self.prompt.format(context=context_text, question=question)

        # Get LLM response
        response = self.llm.invoke(formatted_prompt)
        return response.content

combine_docs_chain = SimpleDocumentChain(llm=llm, prompt=prompt)

print("Document processing chain created")
print("   Chain type: Simple Document Chain")

Document processing chain created
   Chain type: Simple Document Chain


In [12]:
# Create retriever and retrieval chain

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

print("Retriever created from vector store")
print("   Search type: Similarity")
print("   Top-k results: 5")

# Create simple retrieval chain
class SimpleRetrievalChain:
    def __init__(self, retriever, combine_docs_chain):
        self.retriever = retriever
        self.combine_docs_chain = combine_docs_chain

    def invoke(self, input_dict):
        question = input_dict.get("input", "")

        # Retrieve relevant documents
        docs = self.retriever.invoke(question)

        # Process with LLM
        answer = self.combine_docs_chain.invoke({"context": docs, "input": question})

        return {"answer": answer, "context": docs, "input": question}

retrieval_chain = SimpleRetrievalChain(retriever=retriever, combine_docs_chain=combine_docs_chain)

print("\nFull retrieval chain assembled")
print("   Pipeline: Query -> Retriever -> Documents -> LLM -> Response")

Retriever created from vector store
   Search type: Similarity
   Top-k results: 5

Full retrieval chain assembled
   Pipeline: Query -> Retriever -> Documents -> LLM -> Response


In [13]:
# Invoke the retrieval chain to process the user's query

def ask_movie_bot(question: str) -> dict:
    """
    Process a user question through the retrieval chain.
    Returns the response and source documents.
    """
    response = retrieval_chain.invoke({"input": question})
    return response

# Test with sample queries
test_queries = [
    "Recommend some good documentary movies",
    "What are some biography films with high ratings?",
    "Find movies directed by Christopher Nolan",
]

print("🎬 Testing the Movie Chatbot")
print("=" * 60)

for query in test_queries:
    print(f"\n📝 Query: {query}")
    print("-" * 40)
    
    response = ask_movie_bot(query)
    print(f"\n🤖 Response:\n{response['answer']}")
    
    print(f"\n📚 Sources used: {len(response['context'])} documents")
    print("=" * 60)

🎬 Testing the Movie Chatbot

📝 Query: Recommend some good documentary movies
----------------------------------------



🤖 Response:
Here are some highly regarded documentary movies you might enjoy:

1. **The Cove**
   - **Year:** 2009
   - **Genre:** Documentary
   - **Director:** Louie Psihoyos
   - **Star Cast:** Richard O'Barry, Louie Psihoyos, Hardy Jones
   - **IMDb Rating:** 8.4/10
   - **MetaScore:** 84.0
   - **Certificate:** PG-13
   - **Duration:** 92 minutes
   - ![The Cove](https://m.media-amazon.com/images/M/MV5BMzM3NTRhMTctNTE2MS00YjJmLWE2ZmMtMmY3MWI0ODExMTMzXkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_QL75_UY281_CR5,0,190,281_.jpg)

2. **My Enemy's Enemy**
   - **Year:** 2007
   - **Genre:** Documentary
   - **Director:** Kevin Macdonald
   - **Star Cast:** Raymond Aubrac, Robert Badinter, Klaus Barbie
   - **IMDb Rating:** 7.3/10
   - **MetaScore:** 66.0
   - **Certificate:** PG-13
   - **Duration:** 87 minutes
   - ![My Enemy's Enemy](https://m.media-amazon.com/images/M/MV5BMjA5ODQ4NzA1NF5BMl5BanBnXkFtZTYwODU2MDM4._V1_QL75_UY281_CR76,0,190,281_.jpg)

3. **Intent to Destroy: Death, Denial & Depicti


🤖 Response:
Here are some biography films with high ratings from the provided context:

### 1. Schindler's List
- **Year:** 1993
- **Genre:** Biography
- **Director:** Steven Spielberg
- **Star Cast:** Liam Neeson, Ralph Fiennes, Ben Kingsley
- **IMDb Rating:** 9.0/10
- **MetaScore:** 95.0
- **Certificate:** R
- **Duration:** 195 minutes
- **Poster:** ![Schindler's List](https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_QL75_UX190_CR0,2,190,281_.jpg)

### 2. 42
- **Year:** 2013
- **Genre:** Biography
- **Director:** Brian Helgeland
- **Star Cast:** Chadwick Boseman, T.R. Knight, Harrison Ford
- **IMDb Rating:** 7.5/10
- **MetaScore:** 62.0
- **Certificate:** PG-13
- **Duration:** 128 minutes
- **Poster:** ![42](https://m.media-amazon.com/images/M/MV5BMTQwMDU4MDI3MV5BMl5BanBnXkFtZTcwMjU1NDgyOQ@@._V1_QL75_UX190_CR0,0,190,281_.jpg)

### 3. Lincoln
- **Year:** 2012
- **Genre:** Biography
- **Director:** Steven Spielber


🤖 Response:
Here are some movies directed by Christopher Nolan:

1. **The Dark Knight**
   - **Year:** 2008
   - **Genre:** Action
   - **Star Cast:** Jonathan Nolan, Christopher Nolan, David S. Goyer
   - **IMDb Rating:** 9.0/10
   - **MetaScore:** 84.0
   - **Certificate:** PG-13
   - **Duration:** 152 minutes
   - ![Poster](https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_QL75_UX190_CR0,0,190,281_.jpg)

2. **Dunkirk**
   - **Year:** 2017
   - **Genre:** Action
   - **Star Cast:** Fionn Whitehead, Barry Keoghan, Mark Rylance
   - **IMDb Rating:** 7.8/10
   - **MetaScore:** 94.0
   - **Certificate:** PG-13
   - **Duration:** 106 minutes
   - ![Poster](https://m.media-amazon.com/images/M/MV5BN2YyZjQ0NTEtNzU5MS00NGZkLTg0MTEtYzJmMWY3MWRhZjM2XkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_QL75_UX190_CR0,0,190,281_.jpg)

3. **The Dark Knight Rises**
   - **Year:** 2012
   - **Genre:** Action
   - **Star Cast:** Jonathan Nolan, Christopher Nolan, David S. Goyer
   

In [14]:
# Perform adequate formatting to print the final response in a user readable format

def format_movie_response(response: dict, show_sources: bool = False) -> str:
    """
    Format the chatbot response in a user-friendly way.
    
    Args:
        response: The response dict from retrieval chain
        show_sources: Whether to include source movie information
    
    Returns:
        Formatted string response
    """
    output = []
    
    # Main answer
    output.append("🎬 Movie Bot Response:")
    output.append("=" * 50)
    output.append(response['answer'])
    
    # Optionally show source movies
    if show_sources and 'context' in response:
        output.append("\n" + "-" * 50)
        output.append("📚 Movies referenced:")
        
        for i, doc in enumerate(response['context'][:5], 1):
            meta = doc.metadata
            output.append(f"\n  {i}. {meta.get('title', 'N/A')} ({meta.get('year', 'N/A')})")
            output.append(f"     Genre: {meta.get('genre', 'N/A')} | Rating: {meta.get('rating', 'N/A')}/10")
    
    return "\n".join(output)


def chat_with_bot(user_input: str, show_sources: bool = True) -> str:
    """
    Main function to chat with the movie bot.
    """
    response = ask_movie_bot(user_input)
    return format_movie_response(response, show_sources)


# Interactive test
print("💬 Interactive Movie Chatbot Demo")
print("=" * 50)

# Example conversation
queries = [
    "What are some must-watch documentaries?",
    "Recommend a movie with a rating above 8.0"
]

for q in queries:
    print(f"\n👤 You: {q}")
    print(chat_with_bot(q, show_sources=True))
    print()

💬 Interactive Movie Chatbot Demo

👤 You: What are some must-watch documentaries?


🎬 Movie Bot Response:
Here are some must-watch documentaries from the provided list that you might find interesting:

1. **The Salt of the Earth**
   - **Year:** 2014
   - **Genre:** Documentary
   - **Director:** Juliano Ribeiro Salgado
   - **Star Cast:** Sebastião Salgado, Wim Wenders, Lélia Wanick Salgado
   - **IMDb Rating:** 8.4/10
   - **MetaScore:** 83.0
   - **Certificate:** PG-13
   - **Duration:** 110 minutes
   - ![Poster](https://m.media-amazon.com/images/M/MV5BNjkyNjI3ODc0Ml5BMl5BanBnXkFtZTgwMTU0MjM2NDE@._V1_QL75_UX190_CR0,8,190,281_.jpg)

2. **While They Watched**
   - **Year:** 2015
   - **Genre:** Documentary
   - **Director:** Jake J. Smith
   - **Star Cast:** Ahn Chol, Derek Chouinard, Joanna Hosaniak
   - **IMDb Rating:** 7.6/10
   - **MetaScore:** 66.0
   - **Certificate:** PG
   - **Duration:** 92 minutes
   - ![Poster](https://m.media-amazon.com/images/M/MV5BMjM4Mjk2OTE2OV5BMl5BanBnXkFtZTgwMjU5MzMzNjE@._V1_QL75_UX190_CR0,0,190,281_.jpg)

3. **Cave of Forgotten Dr

🎬 Movie Bot Response:
Here are some great movie recommendations with an IMDb rating above 8.0:

1. **Inglourious Basterds**
   - **Year:** 2009
   - **Genre:** Adventure
   - **Director:** Quentin Tarantino
   - **Star Cast:** Brad Pitt, Diane Kruger, Eli Roth
   - **IMDb Rating:** 8.4/10
   - **MetaScore:** 69.0
   - **Certificate:** R
   - **Duration:** 153 minutes
   - ![Inglourious Basterds Poster](https://m.media-amazon.com/images/M/MV5BOTJiNDEzOWYtMTVjOC00ZjlmLWE0NGMtZmE1OWVmZDQ2OWJhXkEyXkFqcGdeQXVyNTIzOTk5ODM@._V1_QL75_UX190_CR0,0,190,281_.jpg)

2. **Seven Samurai**
   - **Year:** 1954
   - **Genre:** Action
   - **Director:** Akira Kurosawa
   - **Star Cast:** Toshirô Mifune, Takashi Shimura, Keiko Tsushima
   - **IMDb Rating:** 8.6/10
   - **MetaScore:** 98.0
   - **Certificate:** Not Rated
   - **Duration:** 207 minutes
   - ![Seven Samurai Poster](https://m.media-amazon.com/images/M/MV5BNTkwY2I5NWMtMjNlNi00ZThjLWI4NzQtNDI4M2I4OGM1YjAzXkEyXkFqcGdeQXVyNzYxODE3NTQ@._V1_QL75_UY2

In [15]:
# Optional: Test the functionality using a Gradio UI

def simple_chat_interface(message: str, history: list) -> str:
    """Simple chat function for Gradio interface."""
    try:
        response = ask_movie_bot(message)
        return response['answer']
    except Exception as e:
        return f"Error: {str(e)}"

# Create a simple Gradio chat interface
simple_demo = gr.ChatInterface(
    fn=simple_chat_interface,
    title="IMDb Movie Chatbot (Test Version)",
    description="Ask me anything about movies!",
    examples=[
        "Recommend a good documentary",
        "What movies has Robert De Niro starred in?",
        "Find me a highly rated biography film",
    ],
)

print("Simple Gradio test interface created")
print("   Uncomment 'simple_demo.launch()' to test the basic chatbot UI")

Simple Gradio test interface created
   Uncomment 'simple_demo.launch()' to test the basic chatbot UI


In [16]:
# Define various agents - each performing a particular task using tool decorator

@tool
def search_movies_by_query(query: str) -> str:
    """
    Search for movies based on a natural language query.
    Use this tool when the user wants to find movies matching certain criteria.
    """
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def get_movie_details(movie_title: str) -> str:
    """
    Get detailed information about a specific movie by title.
    Use this when the user asks about a specific movie.
    """
    query = f"Tell me everything about the movie titled {movie_title}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def recommend_movies_by_genre(genre: str) -> str:
    """
    Recommend movies from a specific genre.
    Use this when the user wants recommendations from a particular genre like Action, Comedy, Drama, etc.
    """
    query = f"Recommend the best {genre} movies with high ratings"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def find_movies_by_actor(actor_name: str) -> str:
    """
    Find movies featuring a specific actor.
    Use this when the user wants to know what movies an actor has appeared in.
    """
    query = f"Find movies starring {actor_name}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def find_movies_by_director(director_name: str) -> str:
    """
    Find movies by a specific director.
    Use this when the user asks about movies from a particular director.
    """
    query = f"Find movies directed by {director_name}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def get_top_rated_movies(min_rating: float = 8.0) -> str:
    """
    Get top rated movies above a certain IMDb rating threshold.
    Use this when the user wants highly rated movies.
    """
    query = f"Find movies with IMDb rating above {min_rating}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


@tool
def compare_movies(movie1: str, movie2: str) -> str:
    """
    Compare two movies.
    Use this when the user wants to compare different movies.
    """
    query = f"Compare the movies {movie1} and {movie2}. What are their ratings, genres, and key differences?"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']


# ============================================================
# POSTER HELPER FUNCTIONS
# ============================================================

def get_movie_posters(movie_titles: List[str], max_posters: int = 5) -> List[Dict[str, str]]:
    """
    Get poster URLs for a list of movie titles.
    
    Args:
        movie_titles: List of movie titles to search for
        max_posters: Maximum number of posters to return
    
    Returns:
        List of dicts with title and poster_url
    """
    posters = []
    
    for title in movie_titles[:max_posters]:
        # Search for the movie in our vector store
        results = vectorstore.similarity_search(f"movie titled {title}", k=1)
        
        if results:
            doc = results[0]
            poster_url = doc.metadata.get('poster_url', '')
            
            if poster_url and poster_url.strip():
                posters.append({
                    'title': doc.metadata.get('title', title),
                    'year': doc.metadata.get('year', ''),
                    'rating': doc.metadata.get('rating', ''),
                    'genre': doc.metadata.get('genre', ''),
                    'poster_url': poster_url
                })
    
    logger.debug(f"Found {len(posters)} posters for {len(movie_titles)} titles")
    return posters


def extract_movie_titles_from_response(response: str) -> List[str]:
    """
    Extract movie titles from a chatbot response.
    Uses simple heuristics to find movie titles.
    """
    import re
    
    titles = []
    
    # Pattern 1: "Movie Title" (YEAR)
    pattern1 = r'"([^"]+)"\s*\((\d{4})\)'
    matches1 = re.findall(pattern1, response)
    titles.extend([m[0] for m in matches1])
    
    # Pattern 2: **Movie Title** (markdown bold)
    pattern2 = r'\*\*([^*]+)\*\*'
    matches2 = re.findall(pattern2, response)
    titles.extend(matches2)
    
    # Pattern 3: Title (YEAR) - Rating
    pattern3 = r'([A-Z][^.!?]*?)\s*\((\d{4})\)\s*-'
    matches3 = re.findall(pattern3, response)
    titles.extend([m[0].strip() for m in matches3])
    
    # Remove duplicates while preserving order
    seen = set()
    unique_titles = []
    for title in titles:
        title_clean = title.strip()
        if title_clean and title_clean.lower() not in seen:
            seen.add(title_clean.lower())
            unique_titles.append(title_clean)
    
    return unique_titles[:5]  # Return top 5 titles


def format_poster_gallery(posters: List[Dict[str, str]]) -> str:
    """
    Format posters as HTML for Gradio display.
    """
    if not posters:
        return ""
    
    html = '<div style="display: flex; flex-wrap: wrap; gap: 15px; margin-top: 15px;">'
    
    for p in posters:
        html += f'''
        <div style="text-align: center; width: 120px;">
            <img src="{p['poster_url']}" 
                 alt="{p['title']}" 
                 style="width: 100px; height: 150px; object-fit: cover; border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.2);"
                 onerror="this.src='https://via.placeholder.com/100x150?text=No+Poster'">
            <p style="font-size: 11px; margin: 5px 0; font-weight: bold;">{p['title'][:20]}{'...' if len(p['title']) > 20 else ''}</p>
            <p style="font-size: 10px; margin: 0; color: #666;">{p['year']} | ⭐ {p['rating']}</p>
        </div>
        '''
    
    html += '</div>'
    return html


# Collect all tools
tools = [
    search_movies_by_query,
    get_movie_details,
    recommend_movies_by_genre,
    find_movies_by_actor,
    find_movies_by_director,
    get_top_rated_movies,
    compare_movies,
]

print("✅ Movie Agent Tools Created:")
print("-" * 40)
for t in tools:
    print(f"  • {t.name}: {t.description[:60]}...")

print("\n✅ Poster display functions created:")
print("  • get_movie_posters: Retrieve poster URLs from database")
print("  • extract_movie_titles_from_response: Parse titles from text")
print("  • format_poster_gallery: Generate HTML gallery")

✅ Movie Agent Tools Created:
----------------------------------------
  • search_movies_by_query: Search for movies based on a natural language query.
Use thi...
  • get_movie_details: Get detailed information about a specific movie by title.
Us...
  • recommend_movies_by_genre: Recommend movies from a specific genre.
Use this when the us...
  • find_movies_by_actor: Find movies featuring a specific actor.
Use this when the us...
  • find_movies_by_director: Find movies by a specific director.
Use this when the user a...
  • get_top_rated_movies: Get top rated movies above a certain IMDb rating threshold.
...
  • compare_movies: Compare two movies.
Use this when the user wants to compare ...

✅ Poster display functions created:
  • get_movie_posters: Retrieve poster URLs from database
  • extract_movie_titles_from_response: Parse titles from text
  • format_poster_gallery: Generate HTML gallery


In [17]:
# Define the orchestrator logic
from collections import deque, OrderedDict

# Simple agent class without external dependencies
class SimpleMovieAgent:
    def __init__(self, tools, llm):
        self.tools = {t.name: t for t in tools}
        self.llm = llm
        self.memory = []

    def invoke(self, input_dict):
        user_input = input_dict.get("input", "")
        # Use the search tool
        result = search_movies_by_query.invoke(user_input)
        self.memory.append({"user": user_input, "assistant": result})
        return {"output": result}

    def clear_memory(self):
        self.memory = []

# Conversation memory placeholder
class ConversationBufferMemory:
    def __init__(self, **kwargs):
        self.messages = []

    def clear(self):
        self.messages = []

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the agent
agent_executor = SimpleMovieAgent(tools, llm)

print("Agent Orchestrator created")
print(f"   - Tools: {len(tools)} specialized movie tools")
print("   - Memory: Conversation buffer for context")

# Test the agent
print("\nTesting Agent Orchestrator...")
print("-" * 50)
test_result = agent_executor.invoke({"input": "Can you recommend some good documentaries?"})
print(f"\nAgent Response:\n{test_result['output']}")

Agent Orchestrator created
   - Tools: 7 specialized movie tools
   - Memory: Conversation buffer for context

Testing Agent Orchestrator...
--------------------------------------------------



Agent Response:
Sure! Here are some great documentaries you might enjoy:

1. **Good Ol' Freda**
   - **Year:** 2013
   - **Genre:** Documentary
   - **Director:** Ryan White
   - **Star Cast:** Freda Kelly, Paul McCartney, John Lennon
   - **IMDb Rating:** 7.4/10
   - **MetaScore:** 60.0
   - **Certificate:** PG
   - **Duration:** 86 minutes
   - ![Good Ol' Freda](https://m.media-amazon.com/images/M/MV5BMjIwMjU3MzUxM15BMl5BanBnXkFtZTcwNzE1NDQ4OQ@@._V1_QL75_UY281_CR1,0,190,281_.jpg)

2. **Boom Bust Boom**
   - **Year:** 2015
   - **Genre:** Documentary
   - **Director:** Bill Jones
   - **Star Cast:** Terry Jones, George Magnus, Robert Shiller
   - **IMDb Rating:** 7.1/10
   - **MetaScore:** 68.0
   - **Certificate:** G
   - **Duration:** 70 minutes
   - ![Boom Bust Boom](https://m.media-amazon.com/images/M/MV5BMTcyOTE4NjAzOV5BMl5BanBnXkFtZTgwNjA3MjExODE@._V1_QL75_UX190_CR0,0,190,281_.jpg)

3. **Cave of Forgotten Dreams**
   - **Year:** 2010
   - **Genre:** Documentary
   - **Director:

In [18]:
# Check the edge cases and handle them appropriately
import time
import hashlib
import json
from collections import deque, OrderedDict

# ============================================================
# QUERY CACHE SYSTEM
# ============================================================

class QueryCache:
    """
    Intelligent query caching system with exact match and semantic similarity.
    Reduces API costs by returning cached responses for similar queries.
    """
    
    def __init__(
        self, 
        max_size: int = 100, 
        ttl_seconds: int = 3600,  # 1 hour default TTL
        similarity_threshold: float = 0.92  # 92% similarity for semantic cache hit
    ):
        """
        Initialize the query cache.
        
        Args:
            max_size: Maximum number of cached entries
            ttl_seconds: Time-to-live for cache entries in seconds
            similarity_threshold: Minimum cosine similarity for semantic cache hit
        """
        self.max_size = max_size
        self.ttl_seconds = ttl_seconds
        self.similarity_threshold = similarity_threshold
        
        # Exact match cache (query hash -> response)
        self.exact_cache: OrderedDict = OrderedDict()
        
        # Semantic cache (stores embeddings for similarity matching)
        self.semantic_cache: List[Dict] = []
        
        # Statistics
        self.stats = {
            "exact_hits": 0,
            "semantic_hits": 0,
            "misses": 0,
            "evictions": 0
        }
        
        logger.info(f"QueryCache initialized: max_size={max_size}, ttl={ttl_seconds}s, threshold={similarity_threshold}")
    
    def _hash_query(self, query: str) -> str:
        """Create a hash for the query string."""
        normalized = query.lower().strip()
        return hashlib.md5(normalized.encode()).hexdigest()
    
    def _is_expired(self, timestamp: float) -> bool:
        """Check if a cache entry has expired."""
        return time.time() - timestamp > self.ttl_seconds
    
    def _evict_if_needed(self):
        """Evict oldest entries if cache is full."""
        while len(self.exact_cache) >= self.max_size:
            self.exact_cache.popitem(last=False)
            self.stats["evictions"] += 1
            logger.debug("Cache eviction performed (exact cache)")
        
        while len(self.semantic_cache) >= self.max_size:
            self.semantic_cache.pop(0)
            self.stats["evictions"] += 1
            logger.debug("Cache eviction performed (semantic cache)")
    
    def _cosine_similarity(self, vec1: List[float], vec2: List[float]) -> float:
        """Calculate cosine similarity between two vectors."""
        import numpy as np
        vec1 = np.array(vec1)
        vec2 = np.array(vec2)
        
        dot_product = np.dot(vec1, vec2)
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        
        if norm1 == 0 or norm2 == 0:
            return 0.0
        
        return dot_product / (norm1 * norm2)
    
    def get_exact(self, query: str) -> Optional[str]:
        """
        Try to get an exact match from cache.
        
        Returns:
            Cached response if found and not expired, None otherwise
        """
        query_hash = self._hash_query(query)
        
        if query_hash in self.exact_cache:
            entry = self.exact_cache[query_hash]
            
            if self._is_expired(entry["timestamp"]):
                # Entry expired, remove it
                del self.exact_cache[query_hash]
                logger.debug(f"Cache entry expired for query hash: {query_hash[:8]}")
                return None
            
            # Move to end (LRU behavior)
            self.exact_cache.move_to_end(query_hash)
            self.stats["exact_hits"] += 1
            logger.info(f"Exact cache hit for query: {query[:50]}...")
            return entry["response"]
        
        return None
    
    def get_semantic(self, query: str, query_embedding: List[float]) -> Optional[str]:
        """
        Try to find a semantically similar cached query.
        
        Args:
            query: The query string
            query_embedding: The embedding vector of the query
            
        Returns:
            Cached response if similar query found, None otherwise
        """
        best_match = None
        best_similarity = 0.0
        expired_indices = []
        
        for i, entry in enumerate(self.semantic_cache):
            # Check expiration
            if self._is_expired(entry["timestamp"]):
                expired_indices.append(i)
                continue
            
            # Calculate similarity
            similarity = self._cosine_similarity(query_embedding, entry["embedding"])
            
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = entry
        
        # Clean up expired entries
        for i in reversed(expired_indices):
            self.semantic_cache.pop(i)
        
        # Check if we have a good enough match
        if best_match and best_similarity >= self.similarity_threshold:
            self.stats["semantic_hits"] += 1
            logger.info(f"Semantic cache hit (similarity: {best_similarity:.3f}) for: {query[:50]}...")
            return best_match["response"]
        
        return None
    
    def set(self, query: str, response: str, query_embedding: Optional[List[float]] = None):
        """
        Store a query-response pair in the cache.
        
        Args:
            query: The query string
            response: The response to cache
            query_embedding: Optional embedding for semantic caching
        """
        self._evict_if_needed()
        
        timestamp = time.time()
        query_hash = self._hash_query(query)
        
        # Store in exact cache
        self.exact_cache[query_hash] = {
            "query": query,
            "response": response,
            "timestamp": timestamp
        }
        
        # Store in semantic cache if embedding provided
        if query_embedding is not None:
            self.semantic_cache.append({
                "query": query,
                "embedding": query_embedding,
                "response": response,
                "timestamp": timestamp
            })
        
        logger.debug(f"Cached response for query: {query[:50]}...")
    
    def clear(self):
        """Clear all cache entries."""
        self.exact_cache.clear()
        self.semantic_cache.clear()
        logger.info("Query cache cleared")
    
    def get_stats(self) -> Dict:
        """Get cache statistics."""
        total_requests = self.stats["exact_hits"] + self.stats["semantic_hits"] + self.stats["misses"]
        hit_rate = (self.stats["exact_hits"] + self.stats["semantic_hits"]) / max(total_requests, 1) * 100
        
        return {
            "exact_cache_size": len(self.exact_cache),
            "semantic_cache_size": len(self.semantic_cache),
            "exact_hits": self.stats["exact_hits"],
            "semantic_hits": self.stats["semantic_hits"],
            "total_hits": self.stats["exact_hits"] + self.stats["semantic_hits"],
            "misses": self.stats["misses"],
            "evictions": self.stats["evictions"],
            "hit_rate_percent": round(hit_rate, 1)
        }


# ============================================================
# RATE LIMITER
# ============================================================

class RateLimiter:
    """
    Simple rate limiter using a sliding window approach.
    """
    
    def __init__(self, max_requests: int = 10, window_seconds: int = 60):
        """
        Initialize rate limiter.
        
        Args:
            max_requests: Maximum number of requests allowed in the time window
            window_seconds: Time window in seconds
        """
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests = deque()
        logger.info(f"Rate limiter initialized: {max_requests} requests per {window_seconds}s")
    
    def is_allowed(self) -> Tuple[bool, Optional[float]]:
        """
        Check if a request is allowed under the rate limit.
        
        Returns:
            Tuple of (is_allowed, wait_time_if_not_allowed)
        """
        now = time.time()
        
        # Remove expired timestamps
        while self.requests and self.requests[0] < now - self.window_seconds:
            self.requests.popleft()
        
        if len(self.requests) < self.max_requests:
            self.requests.append(now)
            return True, None
        else:
            # Calculate wait time until oldest request expires
            wait_time = self.requests[0] + self.window_seconds - now
            logger.warning(f"Rate limit exceeded. Wait time: {wait_time:.1f}s")
            return False, wait_time
    
    def get_remaining(self) -> int:
        """Get remaining requests in current window."""
        now = time.time()
        while self.requests and self.requests[0] < now - self.window_seconds:
            self.requests.popleft()
        return self.max_requests - len(self.requests)


# ============================================================
# MOVIE CHATBOT CLASS (with Caching)
# ============================================================

class MovieChatbot:
    """
    A robust movie chatbot class with edge case handling, logging, rate limiting, and caching.
    """
    
    def __init__(
        self, 
        agent_executor, 
        rate_limit: int = 20, 
        rate_window: int = 60,
        cache_size: int = 100,
        cache_ttl: int = 3600,
        enable_semantic_cache: bool = True
    ):
        """
        Initialize the movie chatbot.
        
        Args:
            agent_executor: The LangChain agent executor
            rate_limit: Max requests per time window (default: 20)
            rate_window: Time window in seconds (default: 60)
            cache_size: Max cached queries (default: 100)
            cache_ttl: Cache TTL in seconds (default: 3600 = 1 hour)
            enable_semantic_cache: Enable semantic similarity caching (default: True)
        """
        self.agent = agent_executor
        self.conversation_history = []
        self.request_count = 0
        self.session_start = time.time()
        self.rate_limiter = RateLimiter(max_requests=rate_limit, window_seconds=rate_window)
        self.cache = QueryCache(max_size=cache_size, ttl_seconds=cache_ttl)
        self.enable_semantic_cache = enable_semantic_cache
        
        logger.info("MovieChatbot instance initialized with caching")
        
    def _validate_input(self, user_input: str) -> Tuple[bool, str]:
        """
        Validate user input and return (is_valid, error_message).
        """
        # Check for empty input
        if not user_input or not user_input.strip():
            logger.warning("Validation failed: Empty input received")
            return False, "Please enter a question or request about movies."
        
        # Check for very short input
        if len(user_input.strip()) < 3:
            logger.warning(f"Validation failed: Input too short ({len(user_input.strip())} chars)")
            return False, "Please provide a more detailed question."
        
        # Check for very long input (potential abuse)
        if len(user_input) > 1000:
            logger.warning(f"Validation failed: Input too long ({len(user_input)} chars)")
            return False, "Your question is too long. Please keep it under 1000 characters."
        
        return True, ""
    
    def _get_query_embedding(self, query: str) -> Optional[List[float]]:
        """Get embedding for a query (for semantic caching)."""
        try:
            return embeddings.embed_query(query)
        except Exception as e:
            logger.warning(f"Failed to get embedding: {e}")
            return None
    
    def chat(self, user_input: str, use_cache: bool = True) -> str:
        """
        Process user input with comprehensive error handling, logging, rate limiting, and caching.
        
        Args:
            user_input: The user's query
            use_cache: Whether to use caching (default: True)
        """
        self.request_count += 1
        request_id = f"REQ-{self.request_count:04d}"
        start_time = time.time()
        
        logger.info(f"[{request_id}] New request received")
        logger.debug(f"[{request_id}] Input: {user_input[:100]}{'...' if len(user_input) > 100 else ''}")
        
        # Validate input first (before checking cache/rate limit)
        is_valid, error_msg = self._validate_input(user_input)
        if not is_valid:
            logger.info(f"[{request_id}] Request rejected: {error_msg}")
            return f"⚠️ {error_msg}"
        
        # Try exact cache first
        if use_cache:
            cached_response = self.cache.get_exact(user_input)
            if cached_response:
                logger.info(f"[{request_id}] Exact cache hit - returning cached response")
                return cached_response
            
            # Try semantic cache
            if self.enable_semantic_cache:
                query_embedding = self._get_query_embedding(user_input)
                if query_embedding:
                    cached_response = self.cache.get_semantic(user_input, query_embedding)
                    if cached_response:
                        logger.info(f"[{request_id}] Semantic cache hit - returning cached response")
                        return cached_response
        
        # Cache miss - check rate limit before making API call
        is_allowed, wait_time = self.rate_limiter.is_allowed()
        if not is_allowed:
            logger.warning(f"[{request_id}] Rate limited. Wait time: {wait_time:.1f}s")
            return f"⚠️ Too many requests. Please wait {wait_time:.0f} seconds before trying again. (Remaining: {self.rate_limiter.get_remaining()})"
        
        # Cache miss recorded
        self.cache.stats["misses"] += 1
        
        try:
            # Process through agent
            logger.debug(f"[{request_id}] Cache miss - invoking agent...")
            response = self.agent.invoke({"input": user_input})
            
            # Calculate response time
            response_time = time.time() - start_time
            
            # Store in history for context
            self.conversation_history.append({
                "request_id": request_id,
                "timestamp": datetime.now().isoformat(),
                "user": user_input,
                "assistant": response['output'],
                "response_time": response_time,
                "cache_hit": False
            })
            
            # Cache the response
            if use_cache:
                query_embedding = self._get_query_embedding(user_input) if self.enable_semantic_cache else None
                self.cache.set(user_input, response['output'], query_embedding)
            
            logger.info(f"[{request_id}] Request completed in {response_time:.2f}s (cached)")
            logger.debug(f"[{request_id}] Response length: {len(response['output'])} chars")
            
            return response['output']
            
        except Exception as e:
            # Calculate error response time
            error_time = time.time() - start_time
            error_type = type(e).__name__
            
            logger.error(f"[{request_id}] Error after {error_time:.2f}s: {error_type} - {str(e)}")
            
            if "RateLimitError" in error_type:
                logger.warning(f"[{request_id}] OpenAI rate limit exceeded")
                return "⚠️ I'm receiving too many requests. Please wait a moment and try again."
            elif "AuthenticationError" in error_type:
                logger.critical(f"[{request_id}] Authentication error - check API key")
                return "⚠️ There's an issue with the API configuration. Please check your API key."
            elif "Timeout" in error_type:
                logger.warning(f"[{request_id}] Request timed out")
                return "⚠️ The request timed out. Please try again with a simpler question."
            else:
                logger.error(f"[{request_id}] Unhandled error: {str(e)}")
                return f"⚠️ An error occurred: {str(e)}. Please try rephrasing your question."
    
    def clear_history(self):
        """Clear conversation history."""
        history_size = len(self.conversation_history)
        self.conversation_history = []
        self.agent.memory.clear()
        logger.info(f"Conversation history cleared ({history_size} messages removed)")
        return "Conversation history cleared!"
    
    def clear_cache(self):
        """Clear the query cache."""
        self.cache.clear()
        return "Query cache cleared!"
    
    def get_history(self) -> List[Dict]:
        """Get conversation history."""
        return self.conversation_history
    
    def get_stats(self) -> Dict:
        """Get session statistics including cache stats."""
        session_duration = time.time() - self.session_start
        cache_stats = self.cache.get_stats()
        
        stats = {
            "session_duration_seconds": round(session_duration, 2),
            "total_requests": self.request_count,
            "conversation_turns": len(self.conversation_history),
            "avg_response_time": round(
                sum(h.get('response_time', 0) for h in self.conversation_history) / 
                max(len(self.conversation_history), 1), 2
            ),
            "rate_limit_remaining": self.rate_limiter.get_remaining(),
            "cache": cache_stats
        }
        logger.debug(f"Session stats: {stats}")
        return stats


# Create the chatbot instance with caching enabled
# - Rate limit: 20 requests per minute
# - Cache: 100 queries, 1 hour TTL
# - Semantic caching: Enabled
chatbot = MovieChatbot(
    agent_executor, 
    rate_limit=20, 
    rate_window=60,
    cache_size=100,
    cache_ttl=3600,
    enable_semantic_cache=True
)

# Test edge cases
print("🧪 Testing Edge Cases and Caching")
print("=" * 50)

edge_cases = [
    "",  # Empty input
    "hi",  # Too short
    "Recommend some good documentaries about nature",  # Valid query
    "Recommend some good documentaries about nature",  # Should hit cache!
    "What's the weather like?",  # Non-movie query (LLM should handle gracefully)
]

for test_input in edge_cases:
    print(f"\n📝 Input: '{test_input}'")
    print(f"🤖 Response: {chatbot.chat(test_input)[:150]}...")
    print("-" * 40)

print("\n✅ Edge case handling, rate limiting, and caching implemented")
print(f"\n📊 Session Stats:")
stats = chatbot.get_stats()
print(f"   Requests: {stats['total_requests']}")
print(f"   Cache Hits: {stats['cache']['total_hits']} (Exact: {stats['cache']['exact_hits']}, Semantic: {stats['cache']['semantic_hits']})")
print(f"   Cache Misses: {stats['cache']['misses']}")
print(f"   Hit Rate: {stats['cache']['hit_rate_percent']}%")

2026-01-18 14:41:49 - INFO - MovieChatbot - Rate limiter initialized: 20 requests per 60s


2026-01-18 14:41:49 - INFO - MovieChatbot - QueryCache initialized: max_size=100, ttl=3600s, threshold=0.92


2026-01-18 14:41:49 - INFO - MovieChatbot - MovieChatbot instance initialized with caching


2026-01-18 14:41:49 - INFO - MovieChatbot - [REQ-0001] New request received


2026-01-18 14:41:49 - WARNING - MovieChatbot - Validation failed: Empty input received


...

2026-01-18 14:42:00 - INFO - MovieChatbot - [REQ-0004] Exact cache hit - returning cached response


2026-01-18 14:42:00 - INFO - MovieChatbot - [REQ-0005] New request received


🤖 Response: Here are some excellent documentaries about nature that you might enjoy:

### 1. **Grizzly Man**
- **Year:** 2005
- **Genre:** Documentary
- **Directo...
----------------------------------------

📝 Input: 'Recommend some good documentaries about nature'
🤖 Response: Here are some excellent documentaries about nature that you might enjoy:

### 1. **Grizzly Man**
- **Year:** 2005
- **Genre:** Documentary
- **Directo...
----------------------------------------

📝 Input: 'What's the weather like?'


2026-01-18 14:42:02 - INFO - MovieChatbot - [REQ-0005] Request completed in 2.22s (cached)


🤖 Response: I'm here to assist with movie recommendations, but I can't provide weather updates. If you're interested in movies, let me know your preferences, and ...
----------------------------------------

✅ Edge case handling, rate limiting, and caching implemented

📊 Session Stats:
   Requests: 5
   Cache Hits: 1 (Exact: 1, Semantic: 0)
   Cache Misses: 2
   Hit Rate: 33.3%


In [19]:
# Create a UI using Gradio
from typing import Iterator, Tuple

last_response_cache = {"response": "", "posters_html": ""}

def agentic_chat(message: str, history: list) -> str:
    """Main chat function for Gradio interface."""
    return chatbot.chat(message)

def clear_conversation():
    """Clear the chatbot conversation history."""
    last_response_cache["response"] = ""
    last_response_cache["posters_html"] = ""
    return chatbot.clear_history()

def get_session_stats():
    """Get current session statistics."""
    stats = chatbot.get_stats()
    cache = stats.get('cache', {})
    return f"""Session Statistics

Requests: {stats['total_requests']}
Conversation Turns: {stats['conversation_turns']}
Avg Response Time: {stats['avg_response_time']:.2f}s
Rate Limit Remaining: {stats['rate_limit_remaining']}
Cache Hit Rate: {cache.get('hit_rate_percent', 0)}%
Session Duration: {stats['session_duration_seconds']:.0f}s"""

# Create the Gradio interface
with gr.Blocks(title="IMDb Movie Chatbot") as demo:
    gr.Markdown("""
    # IMDb Movie Chatbot
    ### Your AI-powered Movie Discovery Assistant

    Ask me anything about movies! I can help you:
    - Find movies by genre, actor, director, or any criteria
    - Get recommendations based on your preferences
    - Compare movies and get detailed information
    """)

    chatbot_ui = gr.ChatInterface(
        fn=agentic_chat,
        examples=[
            "Recommend some highly rated documentaries",
            "Find movies starring Tom Hanks",
            "What are some good adventure movies from the 2000s?",
        ],
    )

    with gr.Row():
        stats_btn = gr.Button("Show Stats")
        stats_output = gr.Textbox(label="Session Stats")
        stats_btn.click(fn=get_session_stats, inputs=[], outputs=[stats_output])

    gr.Markdown("""
    ---
    *Built with LangChain, FAISS, and Gradio*
    """)

print("=" * 60)
print("IMDb Movie Chatbot - Full Version")
print("=" * 60)
print("\nGradio interface created successfully!")
print("\nTo launch the chatbot, uncomment and run:")
print("   demo.launch(share=False)")

IMDb Movie Chatbot - Full Version

Gradio interface created successfully!

To launch the chatbot, uncomment and run:
   demo.launch(share=False)


In [20]:
# Cell 19: TEST SUITE - Run this to test the chatbot

# ============================================================
# TEST CASES
# ============================================================

TEST_CASES = {
    "Basic Functionality": [
        ("BF001", "Genre Search", "Recommend some comedy movies"),
        ("BF002", "Genre Search", "Find documentary films"),
        ("BF003", "Actor Search", "What movies has Tom Hanks starred in?"),
        ("BF004", "Director Search", "Find movies directed by Steven Spielberg"),
        ("BF005", "Rating Filter", "Show movies rated above 8.0"),
        ("BF006", "Year Filter", "What movies came out in 2020?"),
    ],
    "Complex Queries": [
        ("CQ001", "Multi-criteria", "Find a documentary about music with good ratings"),
        ("CQ002", "Time Period", "What adventure movies from the 90s should I watch?"),
        ("CQ003", "Comparison", "Compare documentary and biography genres"),
        ("CQ004", "Recommendation", "Recommend highly rated biography films"),
    ],
    "Edge Cases": [
        ("EC001", "Empty Input", ""),
        ("EC002", "Short Input", "hi"),
        ("EC003", "Non-movie", "What's the weather like?"),
        ("EC004", "Misspelled", "Recomend comdy moveis"),
        ("EC005", "Not Found", "Tell me about movie XYZ123ABC"),
    ],
}

def run_tests(chatbot, verbose=True):
    """Run all test cases and report results."""
    print("=" * 60)
    print("🧪 IMDb MOVIE CHATBOT - TEST SUITE")
    print("=" * 60)
    
    total = 0
    passed = 0
    
    for category, tests in TEST_CASES.items():
        print(f"\n📁 {category}")
        print("-" * 40)
        
        for test_id, test_type, query in tests:
            total += 1
            
            try:
                response = chatbot.chat(query)
                
                # Check if response is valid (not an error for valid queries)
                if query == "":  # Empty should return error
                    success = "⚠️" in response
                elif len(query) < 3:  # Short should return error
                    success = "⚠️" in response
                else:  # Others should return actual content
                    success = len(response) > 20 and "⚠️" not in response
                
                if success:
                    passed += 1
                    status = "✅ PASS"
                else:
                    status = "❌ FAIL"
                
                print(f"\n[{test_id}] {test_type}: {status}")
                print(f"    Query: \"{query[:40]}{'...' if len(query) > 40 else ''}\"")
                
                if verbose:
                    print(f"    Response: {response[:100]}{'...' if len(response) > 100 else ''}")
                    
            except Exception as e:
                print(f"\n[{test_id}] {test_type}: 💥 ERROR")
                print(f"    Error: {str(e)}")
    
    # Summary
    print("\n" + "=" * 60)
    print(f"📊 RESULTS: {passed}/{total} tests passed ({100*passed/total:.1f}%)")
    print("=" * 60)
    
    return passed, total

# Run the tests
print("Running test suite...\n")
passed, total = run_tests(chatbot, verbose=True)

# Quick test examples for manual verification
print("\n" + "=" * 60)
print("📝 ADDITIONAL MANUAL TEST QUERIES")
print("=" * 60)
print("""
Try these queries manually in the Gradio UI:

1. "What are the top 5 highest rated movies?"
2. "Find horror movies from the 2010s"
3. "Movies similar to documentaries about sports"
4. "Who directed the movie Inception?"
5. "List all movies with rating above 8.5"
6. "Compare action and adventure genres"
7. "Find movies less than 90 minutes long"
8. "What PG-13 movies are available?"
""")

2026-01-18 14:42:04 - INFO - MovieChatbot - [REQ-0006] New request received


Running test suite...

🧪 IMDb MOVIE CHATBOT - TEST SUITE

📁 Basic Functionality
----------------------------------------


2026-01-18 14:42:16 - INFO - MovieChatbot - [REQ-0006] Request completed in 12.03s (cached)


2026-01-18 14:42:16 - INFO - MovieChatbot - [REQ-0007] New request received



[BF001] Genre Search: ✅ PASS
    Query: "Recommend some comedy movies"
    Response: Here are some comedy movies you might enjoy:

1. **3 Idiots**
   - **Year**: 2009
   - **Genre**: Co...


...

2026-01-18 14:44:13 - INFO - MovieChatbot - [REQ-0019] Request completed in 17.88s (cached)


2026-01-18 14:44:13 - INFO - MovieChatbot - [REQ-0020] New request received



[EC004] Misspelled: ✅ PASS
    Query: "Recomend comdy moveis"
    Response: Here are some comedy movies you might enjoy:

1. **Filme B - Os Mutantes do Espaço**
   - **Year:** ...


2026-01-18 14:44:14 - INFO - MovieChatbot - [REQ-0020] Request completed in 1.49s (cached)



[EC005] Not Found: ✅ PASS
    Query: "Tell me about movie XYZ123ABC"
    Response: I'm sorry, but it seems that I don't have any information about a movie titled "XYZ123ABC" in the pr...

📊 RESULTS: 15/15 tests passed (100.0%)

📝 ADDITIONAL MANUAL TEST QUERIES

Try these queries manually in the Gradio UI:

1. "What are the top 5 highest rated movies?"
2. "Find horror movies from the 2010s"
3. "Movies similar to documentaries about sports"
4. "Who directed the movie Inception?"
5. "List all movies with rating above 8.5"
6. "Compare action and adventure genres"
7. "Find movies less than 90 minutes long"
8. "What PG-13 movies are available?"



# Test Documentation & Performance Analysis

## Test Categories

### 1. Basic Functionality Tests (BF001-BF006)
| ID | Type | Query | Pass Criteria |
|----|------|-------|---------------|
| BF001 | Genre Search | "Recommend some comedy movies" | Returns comedy movies |
| BF002 | Genre Search | "Find documentary films" | Returns documentaries |
| BF003 | Actor Search | "Movies with Tom Hanks" | Returns actor's films |
| BF004 | Director Search | "Films by Steven Spielberg" | Returns director's work |
| BF005 | Rating Filter | "Movies rated above 8.0" | Returns high-rated films |
| BF006 | Year Filter | "Movies from 2020" | Returns 2020 releases |

### 2. Complex Query Tests (CQ001-CQ004)
| ID | Type | Query | Pass Criteria |
|----|------|-------|---------------|
| CQ001 | Multi-criteria | "Documentary about music with good ratings" | Combines genre + topic + rating |
| CQ002 | Time Period | "Adventure movies from the 90s" | Filters by decade |
| CQ003 | Comparison | "Compare documentary and biography" | Compares two genres |
| CQ004 | Recommendation | "Highly rated biography films" | Quality recommendations |

### 3. Edge Case Tests (EC001-EC005)
| ID | Type | Query | Expected Behavior |
|----|------|-------|-------------------|
| EC001 | Empty Input | "" | Error message |
| EC002 | Short Input | "hi" | Ask for more detail |
| EC003 | Non-movie | "What's the weather?" | Graceful handling |
| EC004 | Misspelled | "Recomend comdy" | Still finds results |
| EC005 | Not Found | "Movie XYZ123ABC" | Says not found |

## Performance Metrics

| Metric | Target | How to Measure |
|--------|--------|----------------|
| Response Time | < 5 seconds | Time from query to response |
| Retrieval Accuracy | Top 5 relevant | Check if results match query |
| Error Handling | 100% graceful | No crashes on edge cases |
| Conversation Memory | Maintains context | Follow-up questions work |

## Future Test Additions
- [ ] Load testing (multiple concurrent users)
- [ ] Response consistency (same query = similar results)
- [ ] Latency benchmarking
- [ ] Token usage optimization
- [ ] Multimodal tests (when poster display added)